In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import requests

In [3]:
weburl = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(weburl,'lxml')

In [4]:
tableCan = soup.find('table',{'class':'wikitable sortable'})
headers="Postcode,Borough,Neighbourhood"
table2=""
for tr in tableCan.find_all('tr'):
    row1=""
    for tds in tr.find_all('td'):
        row1=row1+","+tds.text
    table2=table2+row1[1:]
file=open("toronto.csv","wb")
file.write(bytes(table2,encoding="ascii",errors="ignore"))
df = pd.read_csv('toronto.csv',header=None)
df.columns=["Postalcode","Borough","Neighbourhood"]
indexOut = df[ df['Borough'] =='Not assigned'].index
df.drop(indexOut , inplace=True)
df.loc[df['Neighbourhood'] =='Not assigned' , 'Neighbourhood'] = df['Borough']

In [7]:
joinmore = df.groupby(['Postalcode','Borough'], sort=False).agg( ', '.join)
dffinal=joinmore.reset_index()

In [8]:
!wget -q -O 'Toronto_long_lat_data.csv'  http://cocl.us/Geospatial_data
dflonlat = pd.read_csv('Toronto_long_lat_data.csv')
dflonlat.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [9]:
dflonlat.columns=['Postalcode','Latitude','Longitude']

In [10]:
tordf = pd.merge(dffinal,
                 dflonlat[['Postalcode','Latitude', 'Longitude']],
                 on='Postalcode')

In [11]:
tordf

,Postalcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
7,M3B,North York,Don Mills North,43.745906,-79.352188
8,M4B,East York,"Woodbine Gardens, Parkview Hill",43.706397,-79.309937
9,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937


In [12]:
import random 
!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim 
from IPython.display import Image 
from IPython.core.display import HTML 
from pandas.io.json import json_normalize
!conda install -c conda-forge folium=0.5.0 --yes
import folium 

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    openssl-1.1.1c             |       h516909a_0         2.1 MB  conda-forge
    certifi-2019.9.11          |           py36_0         147 KB  conda-forge
    ca-certificates-2019.9.11  |       hecc5488_0         144 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0         conda-forge
    geopy:           1.20.0-py_0       conda-forge

The following packages will be UPDATED:

    ca-

In [14]:
CLIENT_ID = '0XBJZDANGRODWH3O4IDGTS4PTIUQQAGDBS5QFK1WCMPZM2EW' # your Foursquare ID
CLIENT_SECRET = '5MRPGORJGTYDSOTJWTPRAAPKND51PES11AC5UJJ3YPLKMTOO' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 0XBJZDANGRODWH3O4IDGTS4PTIUQQAGDBS5QFK1WCMPZM2EW
CLIENT_SECRET:5MRPGORJGTYDSOTJWTPRAAPKND51PES11AC5UJJ3YPLKMTOO


# I used Google Maps to find random place in Toronto. 

In [15]:
latitude = 43.761665
longitude = -79.490929
print(latitude, longitude)

43.761665 -79.490929


# It is time for Pizza! I decided to find pizza restaruants/places near this random place/house!

In [19]:
search_query = 'Pizza'
radius = 500

In [20]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=0XBJZDANGRODWH3O4IDGTS4PTIUQQAGDBS5QFK1WCMPZM2EW&client_secret=5MRPGORJGTYDSOTJWTPRAAPKND51PES11AC5UJJ3YPLKMTOO&ll=43.761665,-79.490929&v=20180604&query=Pizza&radius=500&limit=30'

In [21]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5db0aee5c53093002c7a47f5'},
 'response': {'venues': [{'id': '4dd8607be4cd37c893c4497d',
    'name': 'Pizza Nova',
    'location': {'address': '3928 Keele St.',
     'lat': 43.762761957796464,
     'lng': -79.49120791067236,
     'labeledLatLngs': [{'label': 'display',
       'lat': 43.762761957796464,
       'lng': -79.49120791067236}],
     'distance': 124,
     'postalCode': 'M3J 1N8',
     'cc': 'CA',
     'city': 'Toronto',
     'state': 'ON',
     'country': 'Canada',
     'formattedAddress': ['3928 Keele St.', 'Toronto ON M3J 1N8', 'Canada']},
    'categories': [{'id': '4bf58dd8d48988d1ca941735',
      'name': 'Pizza Place',
      'pluralName': 'Pizza Places',
      'shortName': 'Pizza',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/pizza_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1571860197',
    'hasPerk': False},
   {'id': '57c565d1498e511ed9c27695',
    'name': 'Pizza Pizza',
    'loc

In [22]:
venues = results['response']['venues']
dataframe = json_normalize(venues)
dataframe.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId
0,"[{'id': '4bf58dd8d48988d1ca941735', 'name': 'P...",False,4dd8607be4cd37c893c4497d,3928 Keele St.,CA,Toronto,Canada,NaN,124,"[3928 Keele St., Toronto ON M3J 1N8, Canada]","[{'label': 'display', 'lat': 43.76276195779646...",43.762762,-79.491208,M3J 1N8,ON,Pizza Nova,v-1571860197
1,"[{'id': '4bf58dd8d48988d1ca941735', 'name': 'P...",False,57c565d1498e511ed9c27695,1300 FINCH AVENUE WEST,CA,Toronto,Canada,NaN,354,"[1300 FINCH AVENUE WEST, Toronto ON M3J 3K2, C...","[{'label': 'display', 'lat': 43.76474061842327...",43.764741,-79.489785,M3J 3K2,ON,Pizza Pizza,v-1571860197
2,"[{'id': '4bf58dd8d48988d1ca941735', 'name': 'P...",False,4c3b57175810a5935581ba3c,NaN,CA,NaN,Canada,NaN,273,[Canada],"[{'label': 'display', 'lat': 43.7598997761613,...",43.759900,-79.488572,NaN,NaN,Ninos Pizza,v-1571860197
3,"[{'id': '4bf58dd8d48988d1ca941735', 'name': 'P...",False,50ff1716e4b098a6a6203244,NaN,CA,NaN,Canada,Keele and Finch,304,"[Keele and Finch, Canada]","[{'label': 'display', 'lat': 43.764344, 'lng':...",43.764344,-79.490137,NaN,NaN,Regino's Pizza,v-1571860197


In [23]:
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]
dataframe_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Pizza Nova,Pizza Place,3928 Keele St.,CA,Toronto,Canada,NaN,124,"[3928 Keele St., Toronto ON M3J 1N8, Canada]","[{'label': 'display', 'lat': 43.76276195779646...",43.762762,-79.491208,M3J 1N8,ON,4dd8607be4cd37c893c4497d
1,Pizza Pizza,Pizza Place,1300 FINCH AVENUE WEST,CA,Toronto,Canada,NaN,354,"[1300 FINCH AVENUE WEST, Toronto ON M3J 3K2, C...","[{'label': 'display', 'lat': 43.76474061842327...",43.764741,-79.489785,M3J 3K2,ON,57c565d1498e511ed9c27695
2,Ninos Pizza,Pizza Place,NaN,CA,NaN,Canada,NaN,273,[Canada],"[{'label': 'display', 'lat': 43.7598997761613,...",43.759900,-79.488572,NaN,NaN,4c3b57175810a5935581ba3c
3,Regino's Pizza,Pizza Place,NaN,CA,NaN,Canada,Keele and Finch,304,"[Keele and Finch, Canada]","[{'label': 'display', 'lat': 43.764344, 'lng':...",43.764344,-79.490137,NaN,NaN,50ff1716e4b098a6a6203244


In [24]:
dataframe_filtered.name

0        Pizza Nova
1       Pizza Pizza
2       Ninos Pizza
3    Regino's Pizza
Name: name, dtype: object

Let's see where are the pizza places ?

In [25]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) 

folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='My random house',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)


venues_map

# Let's try to make radius bigger! There must be a lot of pizza restaurants around my random house in Toronto!

In [26]:
radius = 5000
url2 = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url2

'https://api.foursquare.com/v2/venues/search?client_id=0XBJZDANGRODWH3O4IDGTS4PTIUQQAGDBS5QFK1WCMPZM2EW&client_secret=5MRPGORJGTYDSOTJWTPRAAPKND51PES11AC5UJJ3YPLKMTOO&ll=43.761665,-79.490929&v=20180604&query=Pizza&radius=5000&limit=30'

In [27]:
results2 = requests.get(url2).json()
results2

{'meta': {'code': 200, 'requestId': '5db0b1aac53093002c89c03a'},
 'response': {'venues': [{'id': '4bc0eda24cdfc9b6a4ca9321',
    'name': '341 Pizza & Wings',
    'location': {'address': 'Four Winds',
     'lat': 43.76346437768995,
     'lng': -79.49847340621493,
     'labeledLatLngs': [{'label': 'display',
       'lat': 43.76346437768995,
       'lng': -79.49847340621493}],
     'distance': 638,
     'cc': 'CA',
     'city': 'Toronto',
     'state': 'ON',
     'country': 'Canada',
     'formattedAddress': ['Four Winds', 'Toronto ON', 'Canada']},
    'categories': [{'id': '4bf58dd8d48988d1ca941735',
      'name': 'Pizza Place',
      'pluralName': 'Pizza Places',
      'shortName': 'Pizza',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/pizza_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1571860906',
    'hasPerk': False},
   {'id': '4dd8607be4cd37c893c4497d',
    'name': 'Pizza Nova',
    'location': {'address': '3928 Keele St.',
    

In [28]:
venues2 = results2['response']['venues']
dataframe2 = json_normalize(venues2)
dataframe2.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId
0,"[{'id': '4bf58dd8d48988d1ca941735', 'name': 'P...",False,4bc0eda24cdfc9b6a4ca9321,Four Winds,CA,Toronto,Canada,NaN,638,"[Four Winds, Toronto ON, Canada]","[{'label': 'display', 'lat': 43.76346437768995...",43.763464,-79.498473,NaN,ON,341 Pizza & Wings,v-1571860906
1,"[{'id': '4bf58dd8d48988d1ca941735', 'name': 'P...",False,4dd8607be4cd37c893c4497d,3928 Keele St.,CA,Toronto,Canada,NaN,124,"[3928 Keele St., Toronto ON M3J 1N8, Canada]","[{'label': 'display', 'lat': 43.76276195779646...",43.762762,-79.491208,M3J 1N8,ON,Pizza Nova,v-1571860906
2,"[{'id': '4bf58dd8d48988d1ca941735', 'name': 'P...",False,57c565d1498e511ed9c27695,1300 FINCH AVENUE WEST,CA,Toronto,Canada,NaN,354,"[1300 FINCH AVENUE WEST, Toronto ON M3J 3K2, C...","[{'label': 'display', 'lat': 43.76474061842327...",43.764741,-79.489785,M3J 3K2,ON,Pizza Pizza,v-1571860906
3,"[{'id': '4bf58dd8d48988d1ca941735', 'name': 'P...",False,4c3b57175810a5935581ba3c,NaN,CA,NaN,Canada,NaN,273,[Canada],"[{'label': 'display', 'lat': 43.7598997761613,...",43.759900,-79.488572,NaN,NaN,Ninos Pizza,v-1571860906
4,"[{'id': '4bf58dd8d48988d1ca941735', 'name': 'P...",False,50ff1716e4b098a6a6203244,NaN,CA,NaN,Canada,Keele and Finch,304,"[Keele and Finch, Canada]","[{'label': 'display', 'lat': 43.764344, 'lng':...",43.764344,-79.490137,NaN,NaN,Regino's Pizza,v-1571860906


In [31]:
filtered_columns = ['name', 'categories'] + [col for col in dataframe2.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe2.loc[:, filtered_columns]
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]
dataframe_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,341 Pizza & Wings,Pizza Place,Four Winds,CA,Toronto,Canada,NaN,638,"[Four Winds, Toronto ON, Canada]","[{'label': 'display', 'lat': 43.76346437768995...",43.763464,-79.498473,NaN,ON,4bc0eda24cdfc9b6a4ca9321
1,Pizza Nova,Pizza Place,3928 Keele St.,CA,Toronto,Canada,NaN,124,"[3928 Keele St., Toronto ON M3J 1N8, Canada]","[{'label': 'display', 'lat': 43.76276195779646...",43.762762,-79.491208,M3J 1N8,ON,4dd8607be4cd37c893c4497d
2,Pizza Pizza,Pizza Place,1300 FINCH AVENUE WEST,CA,Toronto,Canada,NaN,354,"[1300 FINCH AVENUE WEST, Toronto ON M3J 3K2, C...","[{'label': 'display', 'lat': 43.76474061842327...",43.764741,-79.489785,M3J 3K2,ON,57c565d1498e511ed9c27695
3,Ninos Pizza,Pizza Place,NaN,CA,NaN,Canada,NaN,273,[Canada],"[{'label': 'display', 'lat': 43.7598997761613,...",43.759900,-79.488572,NaN,NaN,4c3b57175810a5935581ba3c
4,Regino's Pizza,Pizza Place,NaN,CA,NaN,Canada,Keele and Finch,304,"[Keele and Finch, Canada]","[{'label': 'display', 'lat': 43.764344, 'lng':...",43.764344,-79.490137,NaN,NaN,50ff1716e4b098a6a6203244
5,Pizza Pizza,Pizza Place,800 Unit C-01 Steeles Avenue West,CA,Thornhill,Canada,NaN,4879,"[800 Unit C-01 Steeles Avenue West, Thornhill ...","[{'label': 'display', 'lat': 43.79290649613756...",43.792906,-79.448340,L4J 7L2,ON,4ae4d11bf964a5206f9e21e3
6,Pizza Pizza,Pizza Place,NaN,CA,NaN,Canada,NaN,740,[Canada],"[{'label': 'display', 'lat': 43.76332493356069...",43.763325,-79.499852,NaN,NaN,51198b6be4b02eba04902109
7,Student Pizza & Curry,Pizza Place,NaN,CA,NaN,Canada,NaN,638,[Canada],"[{'label': 'display', 'lat': 43.76347505539495...",43.763475,-79.498462,NaN,NaN,54ff654d498ee5e08c20b2c8
8,Sapore Pizza & Wing,Pizza Place,45 Four Winds Drive,CA,Toronto,Canada,Sentinel Road,637,"[45 Four Winds Drive (Sentinel Road), Toronto ...","[{'label': 'display', 'lat': 43.763762, 'lng':...",43.763762,-79.498303,M3J 1K7,ON,5c51eaa646e1b6002cb6eb3c
9,Pizza Nova,Pizza Place,2210 Jane Street,CA,Toronto,Canada,NaN,4542,"[2210 Jane Street, Toronto ON M3M 1A4, Canada]","[{'label': 'display', 'lat': 43.72306, 'lng': ...",43.723060,-79.509217,M3M 1A4,ON,4bc232f44cdfc9b638c09521


In [32]:
dataframe_filtered.name

0              341 Pizza & Wings
1                     Pizza Nova
2                    Pizza Pizza
3                    Ninos Pizza
4                 Regino's Pizza
5                    Pizza Pizza
6                    Pizza Pizza
7          Student Pizza & Curry
8            Sapore Pizza & Wing
9                     Pizza Nova
10                   Pizza Pizza
11                   Pizza Pizza
12                   Pizza Pizza
13                     Pizza Hut
14                    Pizza Nova
15                   Pizza Pizza
16                    Pizza Nova
17         Express Pizza & Grill
18                Domino's Pizza
19                   Pizza Pizza
20                  Boston Pizza
21              Panzerotto Pizza
22                   Pizza Bella
23                   Pizza Pizza
24    Pie Wood Fired Pizza Joint
25                  Gino's Pizza
26                    Pizza Nova
27          Little Caesars Pizza
28                   Pizza Pizza
29                 Pizza e Pazzi
Name: name

In [33]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) 

folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='My random house',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)


venues_map